In [1]:
# Reload when code changed:
%load_ext autoreload
%autoreload 2
%pwd
import os 
import sys
path = "../"
sys.path.append(path)
#os.path.abspath("../")
print(os.path.abspath(path))

D:\github\ekostat_calculator


In [2]:
import pandas as pd
import numpy as np
import json
import timeit
import time
import core
import importlib
importlib.reload(core)
import logging
importlib.reload(core) 
try:
    logging.shutdown()
    importlib.reload(logging)
except:
    pass
from event_handler import EventHandler
print(core.__file__)
pd.__version__

..\core\__init__.py


'0.19.2'

### Load directories

In [3]:
root_directory = 'D:/github/ekostat_calculator'#"../" #os.getcwd()
workspace_directory = root_directory + '/workspaces' 
resource_directory = root_directory + '/resources'
#alias = 'lena'
user_id = 'test_user' #kanske ska vara off_line user?
workspace_alias = 'lena_indicator'

## Initiate EventHandler

In [4]:
print(root_directory)
paths = {'user_id': user_id, 
         'workspace_directory': root_directory + '/workspaces', 
         'resource_directory': root_directory + '/resources', 
         'log_directory': 'D:/github' + '/log', 
         'test_data_directory': 'D:/github' + '/test_data'}

D:/github/ekostat_calculator


In [5]:
t0 = time.time()
ekos = EventHandler(**paths)
#request = ekos.test_requests['request_workspace_list']
#response = ekos.request_workspace_list(request) 
#ekos.write_test_response('request_workspace_list', response)
print('-'*50)
print('Time for request: {}'.format(time.time()-t0))
# OLD: ekos = EventHandler(root_directory)

2018-08-15 16:23:11,619	logger.py	85	add_log	DEBUG	
2018-08-15 16:23:11,619	logger.py	86	add_log	DEBUG	========================================================================================================================
2018-08-15 16:23:11,619	logger.py	87	add_log	DEBUG	### Log added for log_id "event_handler" at locaton: D:\github\log\main_event_handler.log
2018-08-15 16:23:11,619	logger.py	88	add_log	DEBUG	------------------------------------------------------------------------------------------------------------------------
2018-08-15 16:23:11,619	event_handler.py	74	__init__	DEBUG	Start EventHandler: event_handler


event_handler
D:/github/log
main
----------------------------------------------------------------------------------------------------
--------------------------------------------------
Time for request: 3.6233036518096924


### Load existing workspace

In [6]:
#ekos.copy_workspace(source_uuid='default_workspace', target_alias='lena_1')

In [7]:
ekos.print_workspaces()

Current workspaces for user are:

uuid                                    alias                         status                        
----------------------------------------------------------------------------------------------------
default_workspace                       default_workspace             readable                      
ddc27979-76f8-471c-a0b4-3bc773c6ecbf    lena                          editable                      
9eea0d00-c024-410f-912a-980eed55acae    lena_newdata                  editable                      
147f5d47-773c-43f0-b337-57208718d0cf    lena_indicator                editable                      
6fe05a39-2018-4cba-9dd4-27057578ff23    lena_1                        editable                      
62813e21-5eba-48d0-b460-5254bd0e3be9    Arnold_1                      editable                      


In [8]:
workspace_uuid = ekos.get_unique_id_for_alias(workspace_alias = 'lena_indicator')
print(workspace_uuid)

147f5d47-773c-43f0-b337-57208718d0cf


In [9]:
workspace_alias = ekos.get_alias_for_unique_id(workspace_unique_id = workspace_uuid)

In [10]:
ekos.load_workspace(unique_id = workspace_uuid)
# Här får jag ofta felmeddelande:
# AttributeError: module 'core' has no attribute 'ParameterMapping'

2018-08-15 16:23:16,152	event_handler.py	1674	load_workspace	DEBUG	Trying to load new workspace "147f5d47-773c-43f0-b337-57208718d0cf" with alias "lena_indicator"
2018-08-15 16:23:16,206	logger.py	85	add_log	DEBUG	
2018-08-15 16:23:16,207	logger.py	86	add_log	DEBUG	========================================================================================================================
2018-08-15 16:23:16,223	logger.py	87	add_log	DEBUG	### Log added for log_id "043e3905-31f4-4425-a5ea-eb63e1dfeff7" at locaton: D:\github\ekostat_calculator\workspaces\147f5d47-773c-43f0-b337-57208718d0cf\log\subset_043e3905-31f4-4425-a5ea-eb63e1dfeff7.log
2018-08-15 16:23:16,239	logger.py	88	add_log	DEBUG	------------------------------------------------------------------------------------------------------------------------
2018-08-15 16:23:16,306	logger.py	85	add_log	DEBUG	
2018-08-15 16:23:16,306	logger.py	86	add_log	DEBUG	==================================================================================


Initiating WorkSpace: D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf
Parent directory is: D:/github/ekostat_calculator/workspaces
Resource directory is: D:/github/ekostat_calculator/resources
=== 043e3905-31f4-4425-a5ea-eb63e1dfeff7
!!! period_2007-2012_refvalues_2006
!!! 043e3905-31f4-4425-a5ea-eb63e1dfeff7
!!! D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets
----------------------------------------------------------------------------------------------------
Initiating Subset: D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets/043e3905-31f4-4425-a5ea-eb63e1dfeff7
===
D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets/043e3905-31f4-4425-a5ea-eb63e1dfeff7
load_water_body_station_filter
Initiating WorkStep: D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets/043e3905-31f4-4425-a5ea-eb63e1dfeff7/step_1
load_water_body_stati

2018-08-15 16:23:16,372	logger.py	85	add_log	DEBUG	
2018-08-15 16:23:16,372	logger.py	86	add_log	DEBUG	========================================================================================================================
2018-08-15 16:23:16,372	logger.py	87	add_log	DEBUG	### Log added for log_id "30062c90-2a60-4ee1-9944-f00329db1174" at locaton: D:\github\ekostat_calculator\workspaces\147f5d47-773c-43f0-b337-57208718d0cf\log\subset_30062c90-2a60-4ee1-9944-f00329db1174.log
2018-08-15 16:23:16,372	logger.py	88	add_log	DEBUG	------------------------------------------------------------------------------------------------------------------------
2018-08-15 16:23:16,419	logger.py	85	add_log	DEBUG	
2018-08-15 16:23:16,421	logger.py	86	add_log	DEBUG	========================================================================================================================
2018-08-15 16:23:16,422	logger.py	87	add_log	DEBUG	### Log added for log_id "default_subset" at locaton: D:\github\ekostat_c


Initiating WorkStep: D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets/30062c90-2a60-4ee1-9944-f00329db1174/step_2
load_water_body_station_filter
Initiating WorkStep: D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets/30062c90-2a60-4ee1-9944-f00329db1174/step_3
30062c90-2a60-4ee1-9944-f00329db1174
D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/log
subset
----------------------------------------------------------------------------------------------------
=== default_subset
!!! default_subset
!!! default_subset
!!! D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets
----------------------------------------------------------------------------------------------------
Initiating Subset: D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/subsets/default_subset
===
D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718

2018-08-15 16:23:16,753	logger.py	85	add_log	DEBUG	
2018-08-15 16:23:16,753	logger.py	86	add_log	DEBUG	========================================================================================================================
2018-08-15 16:23:16,769	logger.py	87	add_log	DEBUG	### Log added for log_id "147f5d47-773c-43f0-b337-57208718d0cf" at locaton: D:\github\ekostat_calculator\workspaces\147f5d47-773c-43f0-b337-57208718d0cf\log\workspace_147f5d47-773c-43f0-b337-57208718d0cf.log
2018-08-15 16:23:16,769	logger.py	88	add_log	DEBUG	------------------------------------------------------------------------------------------------------------------------
2018-08-15 16:23:16,769	event_handler.py	1692	load_workspace	INFO	Workspace "147f5d47-773c-43f0-b337-57208718d0cf" with alias "lena_indicator loaded."


147f5d47-773c-43f0-b337-57208718d0cf
D:/github/ekostat_calculator/workspaces/147f5d47-773c-43f0-b337-57208718d0cf/log
workspace
----------------------------------------------------------------------------------------------------


True

In [11]:
#ekos.import_default_data(workspace_alias = workspace_alias)

### Load all data in workspace

In [12]:
#ekos.get_workspace(unique_id = workspace_uuid, alias = workspace_alias).delete_alldata_export()

In [13]:
#%%timeit
ekos.load_data(unique_id = workspace_uuid)

self.all_data 0


2018-08-15 16:23:17,852	workspaces.py	1729	load_all_data	DEBUG	Data has been loaded from existing all_data.pickle file.


In [14]:
w = ekos.get_workspace(unique_id = workspace_uuid)
len(w.data_handler.get_all_column_data_df())

2018-08-15 16:23:18,033	event_handler.py	999	get_workspace	DEBUG	Getting workspace "147f5d47-773c-43f0-b337-57208718d0cf" with alias "lena_indicator"


106094

In [15]:
print('subsetlist', w.get_subset_list())

subsetlist ['043e3905-31f4-4425-a5ea-eb63e1dfeff7', '29329934-9445-4888-b774-f0af30a387bb', '30062c90-2a60-4ee1-9944-f00329db1174', 'default_subset', 'f3ce8113-1065-44da-a4fa-489534fc4e6e']


# Step 0 

In [16]:
w.data_handler.all_data.head()

,AMON,BIOV_CONC_ALL,BQIm,CPHL_BTL,CPHL_INTEG,DEPH,DIN,DOXY_BTL,DOXY_CTD,LATIT_DD,...,TEMP,Q_TEMP,source_TEMP,DOXY,Q_DOXY,source_DOXY,CPHL_INTEG_CALC,CPHL_INTEG_CALC_depths,CPHL_INTEG_CALC_values,index_column
0,NaN,NaN,NaN,NaN,0.68,NaN,NaN,NaN,NaN,58.80317,...,NaN,,,NaN,,,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,0.60,NaN,NaN,NaN,NaN,58.25987,...,NaN,,,NaN,,,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,1.50,NaN,NaN,NaN,NaN,56.37085,...,NaN,,,NaN,,,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,1.02,NaN,NaN,NaN,NaN,65.72950,...,NaN,,,NaN,,,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,1.02,NaN,NaN,NaN,NaN,65.72950,...,NaN,,,NaN,,,NaN,NaN,NaN,4


### Apply first data filter 

In [17]:
w.apply_data_filter(step = 0) # This sets the first level of data filter in the IndexHandler 

# Step 1 
### Set subset filter

In [18]:
#w.copy_subset(source_uuid='default_subset', target_alias='period_2007-2012_refvalues_2013')

In [19]:
subset_alias = 'period_2007-2012_refvalues_2006'
subset_uuid = ekos.get_unique_id_for_alias(workspace_alias = workspace_alias, subset_alias = subset_alias)
#w.set_data_filter(subset = subset_uuid, step=1, 
#                         filter_type='include_list', 
#                         filter_name='MYEAR', 
#                         data=['2007', '2008', '2009', '2010', '2011', '2012']) 


#f1 = w.get_data_filter_object(subset = subset_uuid, step=1) 
#print(f1.include_list_filter)

In [20]:
print('subset_alias:', subset_alias, '\nsubset uuid:', subset_uuid)

subset_alias: period_2007-2012_refvalues_2006 
subset uuid: 043e3905-31f4-4425-a5ea-eb63e1dfeff7


In [21]:
f1 = w.get_data_filter_object(subset = subset_uuid, step=1) 
print(f1.include_list_filter)

{'MYEAR': ['2007', '2008', '2009', '2010', '2011', '2012'], 'STATN': [], 'WATER_BODY': []}


### Apply step 1 datafilter to subset

In [22]:
w.apply_data_filter(subset = subset_uuid, step = 1)

# Step 2
### Load indicator settings filter

In [23]:
w.get_step_object(step = 2, subset = subset_uuid).load_indicator_settings_filters()

### set available indicators

In [24]:
w.get_available_indicators(subset= subset_uuid, step=2)

2018-08-15 16:23:19,937	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7


2018-08-15 16:23:20,215	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2
2018-08-15 16:23:20,306	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2
2018-08-15 16:23:20,371	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7
subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7
subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7
subset

2018-08-15 16:23:20,439	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2
2018-08-15 16:23:20,497	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2
2018-08-15 16:23:20,588	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


...

2018-08-15 16:23:20,718	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2
2018-08-15 16:23:20,821	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7
"['TOT_COVER_ALL'] not in index"
subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7
subset

2018-08-15 16:23:20,924	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2
2018-08-15 16:23:21,060	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2
2018-08-15 16:23:21,181	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


 043e3905-31f4-4425-a5ea-eb63e1dfeff7
subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7
subset 043e3905-31f4-4425-a5ea-eb63e1dfeff7


['din_winter',
 'ntot_summer',
 'ntot_winter',
 'dip_winter',
 'ptot_summer',
 'ptot_winter',
 'bqi',
 'oxygen',
 'biov',
 'chl',
 'secchi']

### Apply indicator data filter

In [25]:
#list(zip(typeA_list, df_step1.WATER_TYPE_AREA.unique()))
#indicator_list = w.get_available_indicators(subset= subset_uuid, step=2)
#indicator_list = ['din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter','bqi','din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter', 'biov', 'chl', 'secchi']
indicator_list = ['din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter']
#indicator_list = ['biov', 'chl']
#indicator_list = ['bqi', 'secchi']

In [26]:
for indicator in indicator_list:
    w.apply_indicator_data_filter(step = 2, 
                          subset = subset_uuid, 
                          indicator = indicator)#,
                         # water_body_list = test_wb)
    #print(w.mapping_objects['water_body'][wb])
    #print('*************************************')

2018-08-15 16:23:21,618	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


Water body SE592000-184700
RESULT [12, 1, 2]
Water body SE581700-113000
RESULT [12, 1, 2, 3]
Water body SE591400-183200
RESULT [12, 1, 2]
Water body SE592040-184000
RESULT [12, 1, 2]
Water body SE561400-161201
RESULT [12, 1, 2]
Water body SE562000-123800
RESULT [12, 1, 2, 3]
Water body SE633000-195000
RESULT [11, 12, 1, 2]
Water body SE625180-181655
RESULT [11, 12, 1, 2]
Water body SE593750-184900
RESULT [12, 1, 2]
Water body SE591200-183600
RESULT [12, 1, 2]
Water body SE592245-184400
RESULT [12, 1, 2]
Water body SE591760-181955
RESULT [12, 1, 2]
Water body SE591400-182320
RESULT [12, 1, 2]
Water body SE591800-181360
RESULT [12, 1, 2]
Water body SE591300-182800
RESULT [12, 1, 2]
Water body SE592290-181600
RESULT [12, 1, 2]
Water body SE592400-180800
RESULT [12, 1, 2]
Water body SE658352-163189
RESULT [12, 1, 2]
Water body SE592468-182000
RESULT [12, 1, 2]
Water body SE592315-182620
RESULT [12, 1, 2]
Water body SE592605-182310
RESULT [12, 1, 2]
Water body SE592420-182210
RESULT [12, 1,

Water body SE651500-213108
RESULT [11, 12, 1, 2]
Water body SE634740-203020
RESULT [11, 12, 1, 2]
Water body SE637310-204860
RESULT [11, 12, 1, 2]
Water body SE630000-183500
RESULT [11, 12, 1, 2]
Water body SE653900-223280
RESULT [11, 12, 1, 2]
Water body SE631710-188130
RESULT [11, 12, 1, 2]
Water body SE632030-187600
RESULT [11, 12, 1, 2]
Water body SE632670-190860
RESULT [11, 12, 1, 2]
Water body SE573300-113801
RESULT [12, 1, 2, 3]
Water body SE561030-122821
RESULT [12, 1, 2, 3]
Water body SE554810-125240
RESULT [12, 1, 2, 3]
Water body SE554040-125750
RESULT [12, 1, 2, 3]
Water body SE552500-124461
RESULT [12, 1, 2]
Water body SE554800-142001
RESULT [12, 1, 2]
Water body SE560825-144215
RESULT [12, 1, 2]
Water body SE561005-150250
RESULT [12, 1, 2]
Water body SE621688-144133
RESULT [12, 1, 2]
Water body SE560900-145280
RESULT [12, 1, 2]
Water body SE560205-143545
RESULT [12, 1, 2]
Water body SE582147-111771
RESULT [12, 1, 2, 3]
Water body SE560940-151740
RESULT [12, 1, 2]
Water bo

Water body SE581240-165220
RESULT [12, 1, 2]
Water body SE581280-170070
RESULT [12, 1, 2]
Water body SE581960-164890
RESULT [12, 1, 2]
Water body SE582590-165000
RESULT [12, 1, 2]
Water body SE621265-173125
RESULT [11, 12, 1, 2]
Water body SE622080-176120
RESULT [11, 12, 1, 2]
Water body SE619690-175690
RESULT [11, 12, 1, 2]
Water body SE621720-175130
RESULT [11, 12, 1, 2]
Water body SE622900-174790
RESULT [11, 12, 1, 2]
Water body SE624380-176450
RESULT [11, 12, 1, 2]
Water body SE594384-185542
RESULT [12, 1, 2]
Water body SE580240-112501
RESULT [12, 1, 2, 3]
Water body SE631500-190270
RESULT [11, 12, 1, 2]
Water body SE622011-146303
RESULT [12, 1, 2]
Water body SE635300-205251
RESULT [11, 12, 1, 2]
Water body SE646360-213700
RESULT [11, 12, 1, 2]
Water body SE643920-211500
RESULT [11, 12, 1, 2]
Water body SE644070-211650
RESULT [11, 12, 1, 2]
Water body SE581820-165500
RESULT [12, 1, 2]
Water body SE650460-213400
RESULT [11, 12, 1, 2]
Water body SE650750-213500
RESULT [11, 12, 1, 2]


2018-08-15 16:23:36,090	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


...

Water body SE650460-213400
RESULT [6, 7, 8]
Water body SE650750-213500
RESULT [6, 7, 8]
Water body SE650280-213110
RESULT [6, 7, 8]
Water body SE562290-124131
RESULT [6, 7, 8]
Water body SE654291-224000
RESULT [6, 7, 8]
Water body SE654200-222920
RESULT [6, 7, 8]
Water body SE654000-222430
RESULT [6, 7, 8]
Water body SE592100-192001
RESULT [6, 7, 8]
Water body SE593500-191660
RESULT [6, 7, 8]
Water body SE593330-192540
RESULT [6, 7, 8]
Water body SE593000-192000
RESULT [6, 7, 8]
Water body SE593500-190000
RESULT [6, 7, 8]
Water body SE560740-144375
RESULT [6, 7, 8]
Water body SE561480-148220
RESULT [6, 7, 8]
Water body SE561150-147620
RESULT [6, 7, 8]
Water body SE612520-172080
RESULT [6, 7, 8]
Water body SE560385-154500
RESULT [6, 7, 8]
Water body SE611766-171305
RESULT [6, 7, 8]
Water body SE603000-181500
RESULT [6, 7, 8]
Water body SE561000-150390
RESULT [6, 7, 8]
Water body SE621157-148904
RESULT [6, 7, 8]
Water body SE583121-171401
RESULT [6, 7, 8]
Water body SE564500-122601
RESUL

2018-08-15 16:24:15,197	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


False
Water body SE590860-113810
waterbody matching file does not recognise water body with VISS_EU_CD SE590860-113810
waterbody matching file does not recognise water body with VISS_EU_CD SE590860-113810
RESULT False
Water body SE580205-165162
RESULT [6, 7, 8]
Water body SE621920-175280
RESULT [6, 7, 8]
Water body SE604055-171248
RESULT [6, 7, 8]
Water body SE590835-183000
RESULT [6, 7, 8]
Water body SE572000-180001
RESULT [6, 7, 8]
Water body SE592500-191750
RESULT [6, 7, 8]
Water body SE592000-190500
RESULT [6, 7, 8]
Water body SE585350-182001
RESULT [6, 7, 8]
--------------------------------------------------
Total time to apply data filters for indicator ptot_summer: 10.128910779953003
--------------------------------------------------
Water body SE592000-184700
RESULT [12, 1, 2]
Water body SE581700-113000
RESULT [12, 1, 2, 3]
Water body SE591400-183200
RESULT [12, 1, 2]
Water body SE592040-184000
RESULT [12, 1, 2]
Water body SE561400-161201
RESULT [12, 1, 2]
Water body SE562000-1

RESULT [12, 1, 2]
Water body SE654575-234250
RESULT [11, 12, 1, 2]
Water body SE654990-224540
RESULT [11, 12, 1, 2]
Water body SE654360-235780
RESULT [11, 12, 1, 2]
Water body SE583875-170270
RESULT [12, 1, 2]
Water body SE652500-213500
RESULT [11, 12, 1, 2]
Water body SE590200-173765
RESULT [12, 1, 2]
Water body SE590740-174135
RESULT [12, 1, 2]
Water body SE584400-172270
RESULT [12, 1, 2]
Water body SE575170-183550
RESULT [12, 1, 2]
Water body SE574170-190001
RESULT [12, 1, 2]
Water body SE594800-190220
RESULT [12, 1, 2]
Water body SE641720-211520
waterbody matching file does not recognise water body with VISS_EU_CD SE641720-211520
waterbody matching file does not recognise water body with VISS_EU_CD SE641720-211520
RESULT False
Water body SE644040-211260
RESULT [11, 12, 1, 2]
Water body SE729159-179002
RESULT [11, 12, 1, 2]
Water body SE651500-213108
RESULT [11, 12, 1, 2]
Water body SE634740-203020
RESULT [11, 12, 1, 2]
Water body SE637310-204860
RESULT [11, 12, 1, 2]
Water body SE6

Water body SE571800-184300
RESULT [12, 1, 2]
Water body SE572350-180930
RESULT [12, 1, 2]
Water body SE580550-112460
RESULT [12, 1, 2, 3]
Water body SE574450-165451
RESULT [12, 1, 2]
Water body SE575000-163620
RESULT [12, 1, 2]
Water body SE561000-153320
RESULT [12, 1, 2]
Water body SE560790-145850
RESULT [12, 1, 2]
Water body SE571000-184001
RESULT [12, 1, 2]
Water body SE634350-202000
waterbody matching file does not recognise water body with VISS_EU_CD SE634350-202000
waterbody matching file does not recognise water body with VISS_EU_CD SE634350-202000
RESULT False
Water body SE623980-175600
RESULT [11, 12, 1, 2]
Water body SE631610-184500
RESULT [11, 12, 1, 2]
Water body SE595000-191501
RESULT [12, 1, 2]
Water body SE631460-185000
RESULT [11, 12, 1, 2]
Water body SE631450-185200
RESULT [11, 12, 1, 2]
Water body SE574160-113351
RESULT [12, 1, 2, 3]
Water body SE575760-112671
RESULT [12, 1, 2, 3]
Water body SE581240-165220
RESULT [12, 1, 2]
Water body SE581280-170070
RESULT [12, 1, 2

# Step 3 
### Set up indicator objects

In [27]:
w.get_step_object(step = 3, subset = subset_uuid).indicator_setup(subset_unique_id = subset_uuid, indicator_list = indicator_list) 

********
din_winter


C:\Anaconda3\envs\LenaEnv\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


waterbody matching file does not recognise water body with VISS_EU_CD SE574050-114780
waterbody matching file does not recognise water body with VISS_EU_CD SE634110-201920
waterbody matching file does not recognise water body with VISS_EU_CD SE634210-202020
waterbody matching file does not recognise water body with VISS_EU_CD SE673283-158060
waterbody matching file does not recognise water body with VISS_EU_CD SE652020-211930
waterbody matching file does not recognise water body with VISS_EU_CD SE634350-202000
waterbody matching file does not recognise water body with VISS_EU_CD SE641720-211520
********
ntot_summer
waterbody matching file does not recognise water body with VISS_EU_CD SE574050-114780
waterbody matching file does not recognise water body with VISS_EU_CD SE634110-201920
waterbody matching file does not recognise water body with VISS_EU_CD SE634210-202020
waterbody matching file does not recognise water body with VISS_EU_CD SE652020-211930
waterbody matching file does not 

C:\Anaconda3\envs\LenaEnv\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


waterbody matching file does not recognise water body with VISS_EU_CD SE574050-114780
waterbody matching file does not recognise water body with VISS_EU_CD SE652020-211930
waterbody matching file does not recognise water body with VISS_EU_CD SE641840-211540
waterbody matching file does not recognise water body with VISS_EU_CD SE673283-158060
waterbody matching file does not recognise water body with VISS_EU_CD SE634210-202020


### CALCULATE STATUS

In [28]:
w.get_step_object(step = 3, subset = subset_uuid).calculate_status(indicator_list = indicator_list)
#'din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter'

2018-08-15 18:22:57,728	workspaces.py	1532	get_filtered_data	DEBUG	STEP: step_2


din_winter


C:\Anaconda3\envs\LenaEnv\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


no data for waterbody SE655120-220380
no data for waterbody SE653140-224000
no data for waterbody SE655180-218660
waterbody matching file does not recognise water body with VISS_EU_CD SE652020-211930
no data for waterbody SE647050-213980
no data for waterbody SE635040-204196
no data for waterbody SE637070-204260
no data for waterbody SE602120-181610
no data for waterbody SE652830-222116
no data for waterbody SE652686-221500
no data for waterbody SE654490-220870
waterbody matching file does not recognise water body with VISS_EU_CD SE574050-114780
no data for waterbody SE570900-164501
waterbody matching file does not recognise water body with VISS_EU_CD SE634210-202020
no data for waterbody SE628750-183300
no data for waterbody SE603650-174500
no data for waterbody SE601020-185050
no data for waterbody SE592600-181600
no data for waterbody SE601070-182870
no data for waterbody SE641840-211540
no data for waterbody SE591815-182670
no data for waterbody SE637640-204160
no data for waterbod

no data for waterbody SE603000-181500
no data for waterbody SE561000-150390
no data for waterbody SE621157-148904
no data for waterbody SE583121-171401
no data for waterbody SE564500-122601
no data for waterbody SE571240-121000
no data for waterbody SE572227-115662
no data for waterbody SE572072-115880
no data for waterbody SE573044-115355
no data for waterbody SE572308-115550
no data for waterbody SE571720-120640
no data for waterbody SE652400-220070
no data for waterbody SE652475-215750
no data for waterbody SE654100-234100
no data for waterbody SE654640-233190
no data for waterbody SE653870-235570
no data for waterbody SE582500-113890
no data for waterbody SE574931-113131
no data for waterbody SE584725-111050
no data for waterbody SE584400-116000
no data for waterbody SE560200-143175
no data for waterbody SE590670-111380
no data for waterbody SE590020-114520
no data for waterbody SE590860-113810
no data for waterbody SE580205-165162
no data for waterbody SE621920-175280
no data for 

waterbody matching file does not recognise water body with VISS_EU_CD SE673283-158060
no data for waterbody SE580150-191251
no data for waterbody SE562450-122751
no data for waterbody SE584200-105901
no data for waterbody SE612303-171075
no data for waterbody SE623300-176210
no data for waterbody SE622795-174565
no data for waterbody SE625416-182696
no data for waterbody SE605660-172380
no data for waterbody SE574450-165451
no data for waterbody SE575000-163620
waterbody matching file does not recognise water body with VISS_EU_CD SE634350-202000
no data for waterbody SE574160-113351
no data for waterbody SE575760-112671
no data for waterbody SE621265-173125
no data for waterbody SE622080-176120
no data for waterbody SE619690-175690
no data for waterbody SE621720-175130
no data for waterbody SE622900-174790
no data for waterbody SE624380-176450
no data for waterbody SE580240-112501
no data for waterbody SE631500-190270
no data for waterbody SE622011-146303
no data for waterbody SE635300

no data for waterbody SE653140-224000
waterbody matching file does not recognise water body with VISS_EU_CD SE652020-211930
no data for waterbody SE647050-213980
no data for waterbody SE602120-181610
no data for waterbody SE652830-222116
no data for waterbody SE652686-221500
no data for waterbody SE654490-220870
waterbody matching file does not recognise water body with VISS_EU_CD SE574050-114780
no data for waterbody SE570900-164501
waterbody matching file does not recognise water body with VISS_EU_CD SE634210-202020
no data for waterbody SE603650-174500
no data for waterbody SE601020-185050
no data for waterbody SE592600-181600
no data for waterbody SE601070-182870
waterbody matching file does not recognise water body with VISS_EU_CD SE641840-211540
no data for waterbody SE591815-182670
no data for waterbody SE585170-175445
no data for waterbody SE652500-213500
no data for waterbody SE590200-173765
waterbody matching file does not recognise water body with VISS_EU_CD SE641720-211520


### CALCULATE QUALITY ELEMENTS

In [29]:
w.get_step_object(step = 3, subset = subset_uuid).calculate_quality_element(subset_unique_id = subset_uuid, quality_element = 'Nutrients', class_name = 'QualityElementNutrients')

********
nutrients
['AMON', 'DIN', 'NTOT', 'NTRA', 'NTRI', 'NTRZ', 'SALT', 'STATIONS_USED', 'STATUS', 'VISS_EU_CD', 'WATER_BODY_NAME', 'WATER_TYPE_AREA', 'YEAR_count', 'global_EQR', 'index_column', 'local_EQR', 'new_index']
['PHOS', 'PTOT', 'SALT', 'STATIONS_USED', 'STATUS', 'VISS_EU_CD', 'WATER_BODY_NAME', 'WATER_TYPE_AREA', 'YEAR_count', 'global_EQR', 'index_column', 'local_EQR', 'new_index']
['DIN', 'NTOT', 'SALT', 'STATIONS_USED', 'STATUS', 'VISS_EU_CD', 'WATER_BODY_NAME', 'WATER_TYPE_AREA', 'YEAR_count', 'global_EQR', 'index_column', 'local_EQR', 'new_index']
['DIN', 'NTOT', 'SALT', 'STATIONS_USED', 'STATUS', 'VISS_EU_CD', 'WATER_BODY_NAME', 'WATER_TYPE_AREA', 'YEAR_count', 'global_EQR', 'index_column', 'local_EQR', 'new_index']
['PHOS', 'PTOT', 'SALT', 'STATIONS_USED', 'STATUS', 'VISS_EU_CD', 'WATER_BODY_NAME', 'WATER_TYPE_AREA', 'YEAR_count', 'global_EQR', 'index_column', 'local_EQR', 'new_index']
['PHOS', 'PTOT', 'SALT', 'STATIONS_USED', 'STATUS', 'VISS_EU_CD', 'WATER_BODY_NAME

In [30]:
w.get_step_object(step = 3, subset = subset_uuid).quality_element['Nutrients'].results[['VISS_EU_CD','WATER_BODY_NAME','WATER_TYPE_AREA','STATUS_NUTRIENTS','mean_EQR','MEAN_N_EQR','EQR_N_winter_mean','global_EQR_ntot_summer','MEAN_P_EQR','EQR_P_winter_mean','global_EQR_ptot_summer'
]].to_csv('D:/Nutrients'+subset_alias+'.txt', float_format='%.3f', header = True, index = None, sep = '\t')

In [31]:
subset_alias

'period_2007-2012_refvalues_2006'

## Plotting results

In [32]:
#%matplotlib inline
#import seaborn as sns
#for name, group in ind_obj.classification_results['status_by_date'].groupby('VISS_EU_CD'):
    #group['date'] = pd.to_datetime(group.SDATE)
    #group.dropna(subset = ['date', 'DIN'])
    #sns.tsplot(data = group.to_dict(), time = 'SDATE', value = 'DIN', condition = 'STATUS', legend = True)
#    group.plot('SDATE', ['DIN', 'REFERENCE_VALUE'], title = name + group.WATER_TYPE_AREA.values[0], marker ='*')

In [33]:
#name + group.WATER_TYPE_AREA.values[0]